<div style="border: none; margin: 5px 0; border-top: 1px dashed #FFFFFF; border-bottom: 1px dashed #FFFFFF; height: 5px;"></div>

<h2 style="color: #FFA07A;">5. Acesso aos serviços no conceito de cidade dos 15 minutos</h2>

In [ ]:
#
import ipywidgets as widgets
import time

# --- Criar um aplicativo HTML para exibir o efeito de escrita ---
output = widgets.HTML(value="<div></div>")
display(output)

# --- Texto formatado para o efeito de escrita ---
texto = """
<div>
    <p> 🔸 Após a explicação sobre a formação dos agrupamentos pelo algoritmo não supervisionado <b>K-means</b>, esta secção apresenta dados que permitem identificar o acesso a serviços no âmbito do conceito de cidade dos 15 minutos, contabilizando o número de <b>residentes com 65 anos ou mais</b> e o número de <b>edifícios</b> em cada opção selecionada.</p> 
    <p>🔸 O conceito de <b><u>cidade dos 15 minutos</u></b> propõe que todos os residentes tenham acesso, a pé ou de bicicleta, a serviços essenciais (como saúde, comércio e lazer) num raio de 15 minutos. Este modelo urbano visa promover uma maior proximidade entre as áreas residenciais e os serviços do quotidiano, favorecendo a sustentabilidade, a coesão social e a qualidade de vida.</p>
    <p>🔸 Além disso, procura reduzir a dependência do transporte motorizado, diminuir as emissões de carbono e fomentar a criação de bairros mais resilientes e inclusivos.</p>
    <p> 🔸 <b>Explore a informação disponibilizada no mapa!</b> </p> 
</div>
"""

# --- Efeito de escrita carácter a carácter (mantém o HTML) ---
texto_html = """
<div style="background-color: #FFFFFF; color: #333333; padding: 15px; 
            border-left: 5px solid #FFA500; font-family: Arial, sans-serif; 
            text-align: justify; font-size: 16px; line-height: 1.6;">
"""

for palavra in texto.split():
    texto_html += palavra + " "
    output.value = texto_html + "</div>"
    time.sleep(0.10) # Ajustar velocidade
    
# --- Garantir que o texto completo é exibido no final ---
output.value = texto_html + "</div>"

In [ ]:
from IPython.display import Javascript, display
# hide-me
display(Javascript('window.cellVisibilityManager.hideCells();'))

# --- Importar as bibliotecas ---
ipython = get_ipython()
ipython.run_line_magic("run", "1.preparacao_bibliotecas.ipynb")

# --- Importar o ficheiro ---
input_pkl_path = "df_servicos_clusters.pkl"
with open(input_pkl_path, 'rb') as pkl_file:
    data = pickle.load(pkl_file)

In [ ]:
# hide-me
display(Javascript('window.cellVisibilityManager.hideCells();'))

df_servicos = data['df']

# --- Garantir que é um GeoDataFrame ---
if not isinstance(df_servicos, gpd.GeoDataFrame):
    df_servicos = gpd.GeoDataFrame(df_servicos, geometry='geometry')

# --- Lista das categorias de serviços consideradas ---
servicos = ['Centro Saude', 'Farmacias', 'Hospitais', 'Supermercados', 'Bancos', 'Parques e jardins', 'CTT']

# --- Contar o número de categorias de serviços distintas por edifício ---
df_servicos['n_categorias_distintas'] = df_servicos[servicos].gt(0).sum(axis=1)

# --- Marcar edifícios que têm acesso a todos os serviços ---
df_servicos['todos_servicos'] = df_servicos.apply(
    lambda row: all(row[servico] > 0 for servico in servicos),
    axis=1
)

# --- Identificar áreas críticas ---
percentil_distancia = df_servicos['distancia_media_servicos'].quantile(0.75)
df_servicos['critico'] = (
    (df_servicos['n_categorias_distintas'] < 5) &
    (df_servicos['distancia_media_servicos'] > percentil_distancia)
)

# --- Funções auxiliares ---
def identificar_areas_baixo_acesso(df, servico):
    return df[df[servico] == 0]

def calcular_populacao_afetada(df):
    if df.empty:
        return 0
    return int(df['pop_64_mais'].sum())

def criar_mapa_interativo(df, cor_borda='white', cor_preenchimento='#999999'):
    mapa = folium.Map(location=[41.1500, -8.6291], 
                      zoom_start=13,
                      min_zoom=13,
                      tiles="CartoDB dark_matter",
                      control_scale=True)

    for _, row in df.iterrows():
        try:
            cor = cor_preenchimento

            tooltip_text = f"""
            <b>População 65+:</b> {int(row['pop_64_mais'])}<br>
            <b>Número de Serviços Próximos:</b> {int(row['numero_servicos_proximos'])}<br>
            <b>Distância Média aos Serviços:</b> {int(row['distancia_media_servicos'])}m<br>
            <b>Serviços:</b><br>
            - Centro Saúde: {int(row['Centro Saude'])}<br>
            - Farmácias: {int(row['Farmacias'])}<br>
            - Hospitais: {int(row['Hospitais'])}<br>
            - Supermercados: {int(row['Supermercados'])}<br>
            - Bancos: {int(row['Bancos'])}<br>
            - Parques e Jardins: {int(row['Parques e jardins'])}<br>
            - CTT: {int(row['CTT'])}
            """
            folium.GeoJson(
                data=row['geometry'].__geo_interface__,
                style_function=lambda x, color=cor: {
                    'fillColor': color,
                    'color': cor_borda,
                    'weight': 0.5,
                    'fillOpacity': 0.6
                },
                tooltip=folium.Tooltip(tooltip_text, sticky=True)
            ).add_to(mapa)
        except Exception as e:
            print(f"[ERRO] Não foi possível adicionar o polígono: {e}")

    return mapa._repr_html_()

# --- Opções e rótulos do menu suspenso ---
dropdown_options = [
    {'label': 'Edifícios', 'value': 'todos'},
    {'label': 'Edifícios com todos os serviços', 'value': 'todos_servicos'},
    {'label': 'Áreas críticas', 'value': 'areas_criticas'},
    {'label': 'Edifícios sem acesso a parques ou jardim', 'value': 'baixo_acesso_Parques e jardins'},
    {'label': 'Edifícios em acesso a supermercado', 'value': 'baixo_acesso_Supermercados'},
    {'label': 'Edifícios sem acesso a hospital', 'value': 'baixo_acesso_Hospitais'},
    {'label': 'Edifícios sem acesso a centro de saúde', 'value': 'baixo_acesso_Centro Saude'},
    {'label': 'Edifícios sem acesso a farmácias', 'value': 'baixo_acesso_Farmacias'},
    {'label': 'Edifícios sem acesso a banco', 'value': 'baixo_acesso_Bancos'},
    {'label': 'Edifícios sem acesso a CTT', 'value': 'baixo_acesso_CTT'},
]

label_opcoes = {opt['value']: opt['label'] for opt in dropdown_options}

# --- Inicializar o painel ---
app = Dash(__name__)
app.index_string = """<!DOCTYPE html>
<html>
    <head>
        {%metas%}
        <title>{%title%}</title>
        {%favicon%}
        {%css%}
        <style>
            html, body {
                margin: 0;
                padding: 0;
                background-color: #111;
                overflow-x: hidden;
                height: 100%;
                border: 1px solid white;
                box-sizing: border-box;
            }
            iframe {
                border: none;
            }
            .custom-dropdown .Select-control,
            .custom-dropdown .Select-menu-outer,
            .custom-dropdown .Select-placeholder,
            .custom-dropdown .Select-value-label {
                white-space: normal !important;
                word-break: break-word !important;
                font-size: 17px;
                min-width: 250px;
                max-width: none;
                width: 100%;
            }
        </style>
    </head>
    <body>
        {%app_entry%}
        <footer>
            {%config%}
            {%scripts%}
            {%renderer%}
        </footer>
    </body>
</html>
"""

# --- Interface da aplicação (filtro em linha) ---
app.layout = html.Div([
    html.H1("Acesso a serviços (~15 minutos)", style={
        'color': 'white', 
        'background-color': '#000', 
        'padding': '10px', 
        'text-align': 'center',
        'margin': '0'
    }),
    
    html.Div([
        html.Label("Selecione a opção para visualizar:",
            style={
                'color': 'white',
                'margin-bottom': '0',
                'margin-right': '10px',
                'whiteSpace': 'nowrap'
            }
        ),
        dcc.Dropdown(
            id='cluster-dropdown',
            options=dropdown_options,
            value='todos',
            clearable=False,
            style={'width': '350px', 'margin-right': '18px'}
        ),
        html.Div(id='info-populacao', style={
            'background-color': '#555',
            'color': 'white',
            'font-size': '16px',
            'border-radius': '5px',
            'padding': '8px 18px',
            'margin-left': 'auto',
            'whiteSpace': 'nowrap',
            'boxShadow': '0 2px 12px #0007'
        })
    ], style={
        'display': 'flex',
        'flex-direction': 'row',
        'align-items': 'center',
        'justify-content': 'center',
        'gap': '10px',
        'margin-top': '10px',
        'margin-bottom': '10px',
        'backgroundColor': '#000',
        'width': '100%'
    }),

    html.Div([
        dcc.Loading(
            id="loading-mapa",
            type="default",  
            color="#007BFF",   
            children=[
                html.Iframe(
                    id='mapa-interativo',
                    srcDoc=None,
                    style={'width': '100%', 'height': '100vh', 'border': 'none'}
                )
            ]
        )
    ], style={'backgroundColor': '#000'})  
])

# --- Callback principal ---
@app.callback(
    [Output('mapa-interativo', 'srcDoc'),
     Output('info-populacao', 'children')],
    [Input('cluster-dropdown', 'value')]
)
def atualizar_mapa(opcao):
    try:
        if opcao == 'todos':
            df_selecionado = df_servicos
            cor_preenchimento = '#999999'
        elif opcao == 'areas_criticas':
            df_selecionado = df_servicos[df_servicos['critico']]
            cor_preenchimento = '#999999'
        elif opcao == 'todos_servicos':
            df_selecionado = df_servicos[df_servicos['todos_servicos']]
            cor_preenchimento = '#999999'
        elif isinstance(opcao, str) and opcao.startswith('baixo_acesso_'):
            servico = opcao.replace('baixo_acesso_', '')
            df_selecionado = identificar_areas_baixo_acesso(df_servicos, servico)
            cor_preenchimento = '#FF0000'
        else:
            return None, "[ERRO] Opção inválida."

        if df_selecionado.empty:
            return None, "[ERRO] Nenhum dado encontrado para essa seleção."

        mapa_html = criar_mapa_interativo(
            df_selecionado,
            cor_borda='white',
            cor_preenchimento=cor_preenchimento
        )

        populacao_afetada = calcular_populacao_afetada(df_selecionado)
        numero_edificios = len(df_selecionado)
        # SOMENTE a informação quantitativa na info-box
        info_texto = f"População 65 anos ou +: {populacao_afetada:,} | Número de edifícios: {numero_edificios:,}"
        return mapa_html, info_texto

    except Exception as e:
        print(f"[ERRO] Não foi possível atualizar o mapa: {e}")
        return None, "[ERRO] Erro interno ao atualizar o mapa."

# --- Encontrar uma porta de rede livre para o painel interativo ---
def find_free_port():
    """Encontra uma porta livre entre 8000 e 9000."""
    while True:
        port = random.randint(8000, 9000)
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            if s.connect_ex(("localhost", port)) != 0:
                return port

port = find_free_port()

# --- Lançar a aplicação interativa quando o ficheiro for executado ---
if __name__ == '__main__':
    app.run(debug=False, port=port)
    
# ---- Mensagem final---
print("\033[92m[INFO] Após análise, pode continuar.\033[0m")

import ipywidgets as widgets
from IPython.display import display

# --- HTML explanatory text widget ---
html_explicacao = widgets.HTML(
    value="""
    <div style="background-color: #FFFFFF; color: #333333; padding: 15px; border-left: 5px solid #6A0DAD; font-family: Arial, sans-serif;"> 
        <p style="text-align: justify;"> 
            <b>A presença dos edifícios no mapa indica se, dentro de um raio de 1.5 km (aproximadamente 15 minutos a pé), estão ou não disponíveis os serviços selecionados</b>. Por exemplo, na opção <b><u>"Área com todos os serviços"</u></b>, os edifícios mostrados no mapa têm todos os serviços considerados a essa distância. Na opção <b><u>"Edifícios sem acesso a Bancos"</u></b>, os edifícios mostrados não têm um banco dentro do raio de ~15 minutos a pé.
        </p>
        <p style="text-align: justify;"> 
            <b>Explore outras opções no mapa! Após essa exploração, responda às questões!</b>
        </p> 
    </div>
    """
)

# --- Display HTML explanation ---
display(html_explicacao)

<div style="border: none; margin: 5px 0; border-top: 1px dashed #FFFFFF; border-bottom: 1px dashed #FFFFFF; height: 5px;"></div>

In [6]:
import os
import pandas as pd
from datetime import datetime
from IPython.display import display, clear_output, Javascript
import ipywidgets as widgets

# hide-me
display(Javascript('window.cellVisibilityManager.hideCells();'))

# --- Pasta específica para guardar as respostas ---
pasta_respostas = ".../respostas"
os.makedirs(pasta_respostas, exist_ok=True)  

# --- Questões de escolha múltipla ---
pergunta1_label = widgets.HTML("<b>4. Quais são os dois serviços menos acessíveis à população com 65 anos ou mais no município do Porto?</b>")
pergunta1 = widgets.RadioButtons(
    options=["a) Farmácia e supermercado.", "b) Banco e CTT.", "c) Hospital e centro de saúde."],
    description='',
    layout=widgets.Layout(width='600px'),
    style={'description_width': 'initial'},
    value=None    
)

pergunta2_label = widgets.HTML("<b>5. Qual é o serviço mais acessível à população com 65 anos ou mais no município do Porto?</b>")
pergunta2 = widgets.RadioButtons(
    options=["a) Banco.", "b) Supermercado.", "c) Farmácia."],
    description='',
    layout=widgets.Layout(width='600px'),
    style={'description_width': 'initial'},
    value=None    
)

# --- Botão para gravar a resposta ---
botao_gravar = widgets.Button(description='Gravar resposta', button_style='info')
output = widgets.Output()

# --- Função para gravar a resposta selecionada ---
def gravar_resposta(b):
    resposta1 = pergunta1.value
    resposta2 = pergunta2.value

        # --- Verificar se ambas as questões foram respondidas ---
    if not all([resposta1, resposta2]):
        with output:
            clear_output()
            print("Por favor, responda a todas as questões antes de gravar.")
        return

     # --- Criar dicionário com as respostas e carimbo temporal ---
    dados = {
        "Resposta 1": [resposta1],
        "Resposta 2": [resposta2],
        "Data": [datetime.now().strftime("%Y-%m-%d %H:%M:%S")]
    }

    # --- Caminho completo para o ficheiro ---
    ficheiro = os.path.join(pasta_respostas, "data.xlsx")
    df = pd.DataFrame(dados)

    # --- Verificar se o ficheiro já existe ---
    if not os.path.isfile(ficheiro):
        with output:
            clear_output()
            print(f"O ficheiro {ficheiro} não existe. Criando o arquivo...")
        df.to_excel(ficheiro, index=False, engine='openpyxl')  # Criar ficheiro, se não existir
        with output:
            clear_output()
            print(f"Ficheiro criado com sucesso em: {ficheiro}")
    else:
        # --- Adicionar nova resposta ao ficheiro existente ---
        df_existente = pd.read_excel(ficheiro, engine='openpyxl')  
        df_final = pd.concat([df_existente, df], ignore_index=True)
        df_final.to_excel(ficheiro, index=False, engine='openpyxl')
        with output:
            clear_output()
            print(f"Resposta adicionada ao ficheiro {ficheiro}")

    # --- Limpar o formulário e mostrar mensagem de sucesso ---
    with output:
        clear_output()
        print("Resposta gravada com sucesso! Obrigado.")
        pergunta1.value = None  # Limpar a primeira pergunta
        pergunta2.value = None  # Limpar a segunda pergunta

# --- Ação ao clicar no botão ---
botao_gravar.on_click(gravar_resposta)

# --- Mostrar instruções e questões ---
display(widgets.HTML("""
<b style="font-size: 18px;">Por favor, responda às questões:</b><br><br>
"""))

# --- Mostrar o formulário ---
display(pergunta1_label, pergunta1, pergunta2_label, pergunta2, botao_gravar, output)

<IPython.core.display.Javascript object>

HTML(value='\n<b style="font-size: 18px;">Por favor, responda às questões:</b><br><br>\n')

HTML(value='<b>4. Quais são os dois serviços essenciais menos acessíveis à população com 65 anos ou mais no mu…

RadioButtons(layout=Layout(width='600px'), options=('a) Centro Saúde e Supermercados.', 'b) Bancos e CTT.', 'c…

HTML(value='<b>5. Qual é o serviço mais acessível à população com 65 anos ou mais no município do Porto?</b>')

RadioButtons(layout=Layout(width='600px'), options=('a) Bancos.', 'b) Supermercados.', 'c) Farmácias.'), style…

Button(button_style='info', description='Gravar resposta', style=ButtonStyle())

Output()

<div style="border: none; margin: 5px 0; border-top: 1px dashed #FFFFFF; border-bottom: 1px dashed #FFFFFF; height: 5px;"></div>

Para prosseguir a análise abra o notebook: [Modelos](6.ipynb)

<div style="border: none; margin: 5px 0; border-top: 1px dashed #FFFFFF; border-bottom: 1px dashed #FFFFFF; height: 5px;"></div>